In [1]:

import os
import pandas as pd
from tqdm import tqdm

import dlib
import cv2 as cv
from mtcnn.mtcnn import MTCNN
import face_recognition as facer

from IPython.display import Image

/data/soft/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
proj = "/root/proj/SecurityAI_Round1"
pimg = f"{proj}/Data/images"
pmdl = f"{proj}/Model"

In [3]:
modelv = "_81"
detector = dlib.get_frontal_face_detector()
%time predictor = dlib.shape_predictor(f"{pmdl}/shape_predictor{modelv}_face_landmarks.dat")


def f_xy(images):
    min_y, max_y, min_x, max_x = 112, 0, 112, 0
    cv_face = detector(cv.cvtColor(images, cv.COLOR_BGR2GRAY), 1)
    if cv_face:
        for face in cv_face:
            shape = predictor(images, face)
            for pt in shape.parts():
                min_y, max_y = min(pt.y, min_y), max(pt.y, max_y)
                min_x, max_x = min(pt.x, min_x), max(pt.x, max_x)
    else:
        min_y, max_y, min_x, max_x = 0, 112, 0, 112
    
    min_y, max_y, min_x, max_x = max(min_y,0), min(max_y,112), max(min_x,0), min(max_x,112)
    return min_y, max_y, min_x, max_x

print(
    f_xy(cv.imread(f"{pimg}/00001.jpg")), 
    f_xy(cv.imread(f"{pimg}/00003.jpg"))
)
%time f_xy(cv.imread(f"{pimg}/00001.jpg"))


CPU times: user 253 ms, sys: 14.9 ms, total: 268 ms
Wall time: 266 ms
(16, 112, 19, 102) (0, 112, 0, 112)
CPU times: user 10.7 ms, sys: 76 µs, total: 10.8 ms
Wall time: 10.4 ms


(16, 112, 19, 102)

In [4]:
%time dmtcnn = MTCNN()

def f_xy_mtcnn(images):
    rdetector = dmtcnn.detect_faces(images)
    min_y, max_y, min_x, max_x = 112, 0, 112, 0
    if rdetector:
        x, y, w, h = rdetector[0]["box"]
        min_y, max_y, min_x, max_x = y, y+h, x, x+w
    else:
        min_y, max_y, min_x, max_x = 0, 112, 0, 112
        
    min_y, max_y, min_x, max_x = max(min_y,0), min(max_y,112), max(min_x,0), min(max_x,112)
    return min_y, max_y, min_x, max_x

print(
    f_xy_mtcnn(cv.imread(f"{pimg}/00001.jpg")), 
    f_xy_mtcnn(cv.imread(f"{pimg}/00003.jpg")),
    f_xy_mtcnn(cv.imread(f"{pimg}/00489.jpg"))
)
%time f_xy_mtcnn(cv.imread(f"{pimg}/00001.jpg"))


CPU times: user 200 ms, sys: 15.9 ms, total: 216 ms
Wall time: 210 ms
(17, 112, 14, 99) (14, 111, 2, 80) (0, 112, 0, 112)
CPU times: user 28.5 ms, sys: 158 µs, total: 28.6 ms
Wall time: 19.7 ms


(17, 112, 14, 99)

In [5]:

def f_xy_facer(images):
    result = facer.face_landmarks(images)
    min_y, max_y, min_x, max_x = 112, 0, 112, 0
    if result:
        resultl = []
        for ir in result[0].values():
            resultl += ir
        for ir in resultl:
            min_y, max_y = min(ir[1], min_y), max(ir[1], max_y)
            min_x, max_x = min(ir[0], min_x), max(ir[0], max_x)
    else:
        min_y, max_y, min_x, max_x = 0, 112, 0, 112
    
    min_y, max_y, min_x, max_x = max(min_y,0), min(max_y,112), max(min_x,0), min(max_x,112)
    return min_y, max_y, min_x, max_x

print(
    f_xy_facer(cv.imread(f"{pimg}/00001.jpg")), 
    f_xy_facer(cv.imread(f"{pimg}/00003.jpg")),
    f_xy_facer(cv.imread(f"{pimg}/00489.jpg"))
)
%time f_xy_facer(cv.imread(f"{pimg}/00004.jpg"))


(36, 112, 19, 102) (0, 112, 0, 112) (0, 112, 0, 112)
CPU times: user 16.9 ms, sys: 91 µs, total: 17 ms
Wall time: 16.6 ms


(38, 112, 0, 86)

In [6]:
lfw = os.listdir(f"{proj}/Data/lfw")
lfw

['Shirley_Jones',
 'Floyd_Keith',
 'Robin_Wright_Penn',
 'Gala_Leon_Garcia',
 'Michael_Olowokandi',
 'Joe_Cocker',
 'William_Cocksedge',
 'Ron_Kirk',
 'Barry_Nakell',
 'Alexandra_Vodjanikova',
 'Nathalie_Gagnon',
 'Abid_Hamid_Mahmud_Al-Tikriti',
 'Sharon_Robinson',
 'Jim_Paxson',
 'Mohammad_Aktar',
 'Mary_Matalin',
 'Leslie_Wiser_Jr',
 'Sigourney_Weaver',
 'Gordon_Lightfoot',
 'David_Arquette',
 'MC_Hammer',
 'Cassandra_Heise',
 'David_Hannay',
 'Jacqueline_Obradors',
 'Iain_Richmond',
 'Mireya_Elisa_Moscoso_Rodriguez',
 'Marc_Leger',
 'Dave_Robertson',
 'Pa_Kou_Hang',
 'Dionigi_Tettamanzi',
 'Jorge_Moreno',
 'Alejandro_Gonzalez_Inarritu',
 'Philip_Murtaugh',
 'Gianna_Angelopoulos-Daskalaki',
 'Valerie_Harper',
 'Arnie_Boehm',
 'Leonard_Hamilton',
 'Dieter_Holzer',
 'Larry_Donald',
 'David_Siegel',
 'Dalia_Rabin-Pelosoff',
 'Robby_Ginepri',
 'Jack_Nicholson',
 'Princess_Diana',
 'James_Layug',
 'Richard_Pennington',
 'Veronica_Lake',
 'Thomas_Weston',
 'Carla_Myers',
 'Oscar_DLeon',
 '

In [7]:
data = pd.read_csv(f"{proj}/Data/securityAI_round1_dev.csv")
datl = data["PersonName"].tolist()
datl


['Paul_McNulty',
 'Claudia_Pechstein',
 'Ranil_Wickremasinghe',
 'Mary-Kate_Olsen',
 'Martina_McBride',
 'Paul_ONeill',
 'Tyron_Garner',
 'Amer_al-Saadi',
 'Alison_Lohman',
 'Duane_Lee_Chapman',
 'Eric_Rosser',
 'David_Wells',
 'Vin_Diesel',
 'Lynne_Cheney',
 'Joseph_Biden',
 'Holly_Hunter',
 'Mathias_Reichhold',
 'George_Lopez',
 'Beth_Jones',
 'Elizabeth_Hurley',
 'Hipolito_Mejia',
 'Charlotte_Rampling',
 'Win_Aung',
 'Robert_Ehrlich',
 'Queen_Elizabeth_II',
 'Grady_Irvin_Jr',
 'Prince_Edward',
 'Jennifer_Aniston',
 'Bob_Huggins',
 'Lou_Piniella',
 'Harbhajan_Singh',
 'Jia_Qinglin',
 'Andy_Roddick',
 'John_Paul_II',
 'Walter_Mondale',
 'Diana_Krall',
 'Penelope_Ann_Miller',
 'Ted_Maher',
 'Tom_Hanks',
 'Luke_Walton',
 'Intisar_Ajouri',
 'Daniel_Day-Lewis',
 'Dick_Cheney',
 'Felix_Mantilla',
 'David_Myers',
 'Jane_Pauley',
 'Chen_Liang_Yu',
 'Gary_Locke',
 'Priscilla_Owen',
 'Miguel_Estrada',
 'Bill_Callahan',
 'Torri_Edwards',
 'Adam_Sandler',
 'Hamzah_Haz',
 'Mesut_Yilmaz',
 'Mike_B

In [9]:
d = 6
ed = 720

fx1 = lambda x1, x2, x3: max(x1,x2)
fx2 = lambda x1, x2, x3: min(x1,x2)


def min_max_x_y(images):
    min_y1, max_y1, min_x1, max_x1 = f_xy(images)
    min_y2, max_y2, min_x2, max_x2 = f_xy_mtcnn(images)
    min_y3, max_y3, min_x3, max_x3 = f_xy_facer(images)
    
    min_y, max_y = fx1(min_y1,min_y2,min_y3), fx2(max_y1,max_y2,max_y3)
    min_x, max_x = fx1(min_x1,min_x2,min_x3), fx2(max_x1,max_x2,max_x3)
    return min_y, max_y, min_x, max_x


for p1 in tqdm(lfw):
    if p1 not in datl:
        try:
            for p2 in os.listdir(f"{proj}/Data/lfw/{p1}")[:1]:
                img = cv.imread(f"{proj}/Data/lfw/{p1}/{p2}")
                img = cv.resize(img, (112, 112))

                min_y, max_y, min_x, max_x = min_max_x_y(img)
                img = img[max(min_y-d,0):min(max_y+d,112), max(min_x-d,0):min(max_x+d,112)]
                img = cv.resize(img, (112, 112))
                
                ed += 1
                op = f"{pimg}/"+f"00000{ed}"[-5:]+".jpg"
                cv.imwrite(op, img)
                # print(op, p1, min_max_x_y(img))
                
        except Exception as e:
            pass



100%|██████████| 5749/5749 [04:29<00:00, 21.32it/s]
